# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [130]:
!pip install selenium # biblioteca muito utilizada como interface com o navegador, ela funciona como um robô que clica, insere dados, etc em páginas WEB: interagindo com os sites como se fosse você.


In [131]:
!pip install selenium==2.45

In [161]:
from selenium import webdriver # permite criar o navegador
from selenium.webdriver.common.keys import Keys # permite escrever no navegador
from selenium.webdriver.common.by import By # permite selecionar itens no navegador

# Configure o Safari para ativar o suporte ao WebDriver -> Na barra acima no Safari escolha -> Desenvolvedor > Permitir Automação Remota.

# abrir um navegador
navegador = webdriver.Safari()


# Passo 1: Pegar a cotação do Dólar

navegador.get("https://www.melhorcambio.com/dolar-hoje")

cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_dolar = cotacao_dolar.replace(",", ".")
print(cotacao_dolar)


# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.melhorcambio.com/euro-hoje")

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_euro = cotacao_euro.replace(",", ".")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.22
5.11
289.00


### Agora vamos atualizar a nossa base de preços com as novas cotações
- Importando a base de dados

In [162]:
# Passo 4: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [163]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.22,5219.9478,1.40,7307.92692
1,Carro Renault,4500.00,Euro,5.11,22995.0000,2.00,45990.00000
2,Notebook Dell,899.99,Dólar,5.22,4697.9478,1.70,7986.51126
3,IPhone,799.00,Dólar,5.22,4170.7800,1.70,7090.32600
4,Carro Fiat,3000.00,Euro,5.11,15330.0000,1.90,29127.00000
5,Celular Xiaomi,480.48,Dólar,5.22,2508.1056,2.00,5016.21120
6,Joia 20g,20.00,Ouro,289.00,5780.0000,1.15,6647.00000


### Agora vamos exportar a nova base de preços atualizada


In [164]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)